In [ ]:
# Copyright (C) 2022 Mila - Institut québécois d'intelligence artificielle
# SPDX-License-Identifier: Apache-2.0

In [ ]:
# Anomaly viewer
#
# Notebook to visualize the original ("raw") dataset locally.
#
# Description of the anomaly viewer:
# - Identification (scrolling menue): Select an anomaly given the anomaly identification.
# - Image (scrolling menue): Select an image that contains the selected anomaly from
#                            the list of images that contains the selected anomaly.
# - The table shows the details of the anomaly present in selected image.
# - The image shows the cable with the bounding box annotations of the anomalies present in the image.

In [ ]:
import sys

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import display
from ipywidgets import Box, VBox, widgets
from PIL import Image, ImageDraw
from preprocess_dataset import compute_polygon_coordinates

In [ ]:
# To adapt
root_directory = os.environ["HOME"]

In [ ]:
data_path = os.path.join(root_directory, "CableInspect-AD")
df = pd.read_csv(os.path.join(data_path, "labels.csv"))
df.dropna(subset=["identification"], inplace=True)

In [ ]:
%matplotlib widget
plt.ioff()
fig = plt.figure(figsize=(8, 4), num="  ")
plt.ion()

# Anomaly selector
anomaly_selector = widgets.Dropdown(options=sorted(df["identification"].unique()), description="Identification: ")


def change_anomaly_selector(*args):
    """Change anomaly selector."""
    dataframe_anomaly = df[df["identification"] == args[0]["new"]]
    img_selector.options = sorted(dataframe_anomaly["image_path"].unique())


anomaly_selector.observe(change_anomaly_selector, "value")

# Image selector
df_anomaly = df[df["identification"] == anomaly_selector.value]
img_selector = widgets.Dropdown(options=sorted(df_anomaly["image_path"].unique()), description="Image: ")


def enable_box(is_enabled, box):
    """Enable box.

    Args:
        is_enabled (bool): Whether or not to enable box.
        box (VBox): VBox widget
    """
    for c in box.children:
        if isinstance(c, Box):
            enable_box(is_enabled, c)
        else:
            c.disabled = not is_enabled


def change_img_selector(*args):
    """Change image selector."""
    enable_box(False, final_box)
    image_path = args[0]["new"]
    image_info = df[df["image_path"] == image_path]
    # Update image
    image = draw_annotated_image(image_path, image_info)
    im.set_data(image)
    # Update image meta-data
    display_info(image_info)
    enable_box(True, final_box)


img_selector.observe(change_img_selector, "value")


def draw_annotated_image(image_path, image_info):
    """Draw annotated image.

    Args:
        image_path (str): Image path.
        image_info (pd.DataFrame): Dataframe containing image meta-data.

    Returns:
        image (PIL.Image): Annotated image.
    """
    image = Image.open(os.path.join(data_path, image_path))
    # Clear text
    for text in plt.gca().texts:
        text.set_visible(False)
    # Draw image
    draw = ImageDraw.Draw(image)
    bbox_col = ["bbox_x", "bbox_y", "bbox_width", "bbox_height", "bbox_rotation"]
    for _, row in image_info.iterrows():
        identification = row["identification"]
        color = "red" if (identification == anomaly_selector.value) else "blue"
        polygon = compute_polygon_coordinates(tuple(row[bbox_col]))
        draw.polygon(xy=polygon, outline=color, fill=None, width=5)
        plt.text(polygon[0][0], polygon[0][1] - 20, identification, color=color, fontsize=18)
    return image


# Pre load image
img_path = img_selector.value
img_info = df[df["image_path"] == img_path]
img = draw_annotated_image(img_path, img_info)
im = plt.imshow(img)


def display_info(image_info):
    """Display image info.

    Args:
        image_info (pd.DataFrame): Dataframe containing image meta-data.
    """
    columns = ["identification", "anomaly_type", "anomaly_grade", "cable_id", "side_id", "pass_id"]
    with info_out:
        info_out.clear_output()
        display(image_info[columns].sort_values(by=["identification"]).style.hide(axis="index"))


# Pre load image meta-data
info_out = widgets.Output()
display_info(img_info)

# Display everything
controls_vbox = VBox([anomaly_selector, img_selector])
display_vbox = VBox([info_out, fig.canvas])
final_box = VBox([controls_vbox, display_vbox])
display(final_box)